# Introduction

This notebook is used by Decision Optimization (DO) model `PredictiveMaintenance` to solve optimization scenarios.  

The model takes into account predicted failure day for each machine, planned production numbers, cost of maintenance if performed before a machine fails, cost of repair if machine fails before maintenance, cost of lost production due to maintenance or repair, any opportunity cost of scheduling maintenance earlier than the machine's remaining life, and maintenance crew availability. It schedules maintenance for each machine in order to maximize production output while minimizing all costs.

# Load and Preprocess Input Data

Load input data from scenario input tables. `predicted_failure` input table contains predictions from our linear regression model for which day a machine is going to fail.  `Fixed maintenance` is used to specify any manually chosen (fixed) days for maintenance that should be taken into account by the optimization model.  For more detailed descriptions of the inputs, please refer to the project README file.

In [27]:
# The code was removed by Watson Studio for sharing.

,id
0,Day-01
1,Day-02
2,Day-03
3,Day-04
4,Day-05


In [28]:

body = client_b1267f57e7aa4cd1b558205e90b442cf.get_object(Bucket='predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru',Key='machine.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_machine = pd.read_csv(body)
df_machine.head()


,id,capacity,remaining life,cost of maintenance,maintenance loss,cost of repair,repair loss,loss per life day,production value unit
0,M-01,100,14,50,50,100,100,20,10
1,M-02,120,12,50,50,100,100,20,10
2,M-03,140,12,50,50,100,100,20,10
3,M-04,140,9,50,50,100,100,20,10
4,M-05,90,10,50,50,100,100,20,10


In [29]:

body = client_b1267f57e7aa4cd1b558205e90b442cf.get_object(Bucket='predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru',Key='predicted_failure.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_predicted_failure = pd.read_csv(body)
df_predicted_failure = df_predicted_failure.set_index(['machine', 'day'])
df_predicted_failure.head()


failure
machine day            
M-01    Day-01        0
        Day-02        0
        Day-03        0
        Day-04        0
        Day-05        0

In [30]:

body = client_b1267f57e7aa4cd1b558205e90b442cf.get_object(Bucket='predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru',Key='fixed_maintenance.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_fixed_maintenance = pd.read_csv(body)
df_fixed_maintenance = df_fixed_maintenance.set_index(['machine', 'day'])
df_fixed_maintenance.head()


maintenance
machine day                
M-01    Day-01            0
        Day-02            0
        Day-03            0
        Day-04            0
        Day-05            0

In [31]:

body = client_b1267f57e7aa4cd1b558205e90b442cf.get_object(Bucket='predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru',Key='parameters.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_parameters = pd.read_csv(body)
df_parameters.head()


,id,value
0,strategy,1
1,maintenance crew size,2
2,fix_maintenance,1


In [32]:

body = client_b1267f57e7aa4cd1b558205e90b442cf.get_object(Bucket='predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru',Key='planned_production.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_planned_production = pd.read_csv(body)
df_planned_production = df_planned_production.set_index(['machine', 'day'])
df_planned_production.head()


production
machine day               
M-01    Day-01         100
        Day-02          96
        Day-03          97
        Day-04         100
        Day-05          77

In [33]:
import pandas as pd
import numpy as np

all_machines = df_machine['id'].values

all_days = df_day['id'].values

`cumul_failure` stands for *cumulative failure* and defines the number of times a machine has failed *before* a given day. To compute cumulative failure for day `i` we sum up the number of failures that have occurred before day `i`.

In [34]:
data_cumul_failure = []
for machine in all_machines:
    for i, d in np.ndenumerate(all_days):
        cumul = 0
        for i2, d2 in np.ndenumerate(all_days):
            if i2==i:
                break
            cumul += int(df_predicted_failure.failure[machine, d2])
        data_cumul_failure.append((machine, d, cumul))

df_cumul_failure = pd.DataFrame(data_cumul_failure, columns=['machine', 'day', 'cumul_failure'])
df_cumul_failure=df_cumul_failure.set_index(['machine', 'day'])

# Implement Optimization Model


## Initialize a CPLEX environment and create a new model object

In [35]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()    

* system is: Linux 64bit
* Python version 3.6.8, located at: /opt/conda/envs/Python36/bin/python
* docplex is present, version is (2, 10, 150)
* CPLEX library is present, version is 12.9.0.1, located at: /opt/conda/envs/Python36/lib/python3.6/site-packages
* pandas is present, version is 0.24.1


In [36]:
from docplex.mp.model import Model
mdl = Model(name="PredictiveMaintenance")

## Define decision variables

Decision variables are the decisions we are trying to make, i.e. the unknowns that the optimization model will assign values to.

`production[m][d]` is an continuous variable that defines how many units of product will be produced on machine `m` on day `d`.

`maintenance[m][d]` is a binary variable that defines whether machine `m` is scheduled for maintenance on day `d`

In [37]:
production = mdl.continuous_var_matrix(keys1=all_machines, keys2=all_days, name=lambda ns: "Production_%s_%s" % (ns[0],ns[1]))
df_production = pd.DataFrame({'production': production})
df_production.index.names=['all_machines', 'all_days']

maintenance = mdl.binary_var_matrix(keys1=all_machines, keys2=all_days, name=lambda ns: "Maintenance_%s_%s" % (ns[0],ns[1]))
df_maintenance = pd.DataFrame({'maintenance': maintenance})
df_maintenance.index.names=['all_machines', 'all_days']

## Define constraints

Constraints are the rules we must respect when making decisions, for example capacity limits or maintenance crew availability

### Take into account fixed maintenance events

If any maintenance events have already been planned, these events have to be accounted for by the optimization model.  

If `fixed_maintenance[m][d] == 1`, then the decision variable `maintenance[m][d]` must also be assigned the value of 1.

In [38]:
fixm = int(df_parameters[df_parameters.id=='fix_maintenance']['value'])
if fixm == 1:
    for machine in all_machines:
        for day in all_days:
            if (df_fixed_maintenance.maintenance[machine, day] == 1):
                mdl.add_constraint(maintenance[machine, day] == 1)

### Take into account reduced production capacity due to maintenance events

The number of units produced on machine `m` on day `d` (`production[m][d]`) is defined as follows:

   - If planned production number is lower than even reduced capacity, due to potential maintenance on that day, add a more generic constraint
   
   `production[m][d] == prod[m][d]` where `prod[m][d]` is the planned production on machine `m` on day `d`.  In this case, a possible maintenance event will not result in reduced production numbers on that day on this machine
   
   
   - Otherwise account for reduced capacity due to a potential maintenance event and make sure assigned production number does not exceed reduced capacity:
   
   `production[m][d] == prod[m][d] - (prod[m][d] - capacity[m][d]*(1-maintenance_loss_fraction))*maintenance[m][d]` where maintenance_loss_fraction is the percentage of production volume lost due to a maintenance event on that day.

In [39]:
for machine in all_machines:       
    maintenance_loss = int(df_machine[df_machine.id==machine]['maintenance loss'])/100.
    capacity = int(df_machine[df_machine.id==machine]['capacity'])
    for day in all_days:   
        prod = df_planned_production.production[machine, day]
        if (prod <= capacity*(1-maintenance_loss)):
            mdl.add_constraint( production[machine, day] == prod )
        else:
            mdl.add_constraint( production[machine, day] == prod - (prod-capacity*(1-maintenance_loss))*maintenance[machine, day])

### Each machine should be scheduled for exactly one maintenance event

In [40]:
for machine in all_machines:       
    mdl.add_constraint( mdl.sum(maintenance[machine, day] for day in all_days) == 1)

### Take into account maintenance crew availability

The number of maintenance crews available is provided as a parameter `maintenance crew size`.  We must define a constraint for each day to make sure that the number of machines scheduled for maintenance on any given day does not exceed `maintenance crew size`.

In [41]:
maintenance_crew_size = int(df_parameters[df_parameters.id=='maintenance crew size']['value'])

for day in all_days:       
    mdl.add_constraint( mdl.sum(maintenance[machine, day] for machine in all_machines) <= maintenance_crew_size)

## KPIs 
Here we define several key KPIs that we can use to compare optimization scenarios. KPIs are *decision expressions*, expressed as function of decision variables, and are available as part of the solution, along with the decision variable values.

### KPI 1: Cost of the solution

The total cost of the solution, `cost`, is comprised of a number of different components.  When computing the potential cost of scheduling maintenance/repair for a machine on day `day2`, we must take into account the following:

- Production lost due to a machine breaking before maintenance. If the number of units of planned production on day `day2` is greater than the reduced production capacity, due to performed repair (i.e. `production_day2 > capacity*(1-repair_loss)`), we must take into account the opportunity cost of the units never produced: `production_value_unit*prob_break_day2*(production_day2 - capacity*(1-repair_loss))` where `prob_break_day2` is the probability that the machine will break on `day2`.

- Higher cost of repair if a machine breaks before planned maintenance: `cost_of_repair*prob_break_before` where `prob_break_before` is the total probability that the machine breaks before `day2`

- Cost of the maintenance procedure if the machine hasn't failed by `day2`: `cost_of_maintenance*(1-prob_break_before)`

- Potential cost of lost production due to a maintenance event.  If the number of units of planned production on day `day2` is greater than the reduced production capacity, due to performed maintenance (i.e. `production_day > capacity*(1-maintenance_loss)`), we must take into account the opportunity cost of the units never produced: `production_value_unit*(production_day - capacity*(1-maintenance_loss))`

- Cost of maintenance performed too early: `loss_per_life_day*lost_days` where `lost_days` is the number of machine life days remaining (before required maintenance)


In [42]:
data_cost_maintenance = []
data_cost_maintenance_detail = []
cost_kpis = []

for machine in all_machines:  
    # obtain machine characteristics
    # for detailed descriptions of the data, please refer to the README file in the project
    life = int(df_machine[df_machine.id==machine]['remaining life'])
    capacity = int(df_machine[df_machine.id==machine]['capacity'])
    cost_of_maintenance = int(df_machine[df_machine.id==machine]['cost of maintenance'])
    # fraction of production capacity lost due to maintenance
    maintenance_loss = int(df_machine[df_machine.id==machine]['maintenance loss'])/100.
    cost_of_repair = int(df_machine[df_machine.id==machine]['cost of repair'])
    # fraction of production capacity lost due to repair
    repair_loss = int(df_machine[df_machine.id==machine]['repair loss'])/100.
    loss_per_life_day = int(df_machine[df_machine.id==machine]['loss per life day'])
    production_value_unit = int(df_machine[df_machine.id==machine]['production value unit'])
    
    previous_day = None
    for i, day in np.ndenumerate(all_days):
        cost, prob_break_before, fail_prod_cost, repair_cost, maint_cost, prod_cost, early_maint_cost = 0, 0, 0, 0, 0, 0, 0;
        
        if (previous_day != None):
            # probability that the machine will break before day `day`
            prob_break_before = int(df_cumul_failure.cumul_failure[machine, previous_day])/100.
        previous_day = day
        
        # Expected cost of lost production if machine fails before maintenance
        # Here we must sum up over all possible days when the machine actually fails so we can account for all lost days between day2,
        # when the machine fails, and day when the repair is scheduled.
        for i2, day2 in np.ndenumerate(all_days):
            if (i2==i):
                break
            prob_break_day2 = int(df_predicted_failure.failure[machine, day2])/100.
            production_day2 = int(df_planned_production.production[machine, day2])
            if (production_day2 > capacity*(1-repair_loss)):
                cur_day_prodloss = production_value_unit*prob_break_day2*(production_day2 - capacity*(1-repair_loss))
                fail_prod_cost += cur_day_prodloss
                cost += cur_day_prodloss
                
        # Cost of repair if machine fails before maintenance
        repair_cost = cost_of_repair*prob_break_before
        cost += repair_cost
        
        # Cost of maintenance, if machine doesn't break before maintenance
        maint_cost = cost_of_maintenance*(1-prob_break_before)
        cost += maint_cost
        
        # Cost of lost production on the day of a maintenance event
        production_day = int(df_planned_production.production[machine, day])
        if (production_day > capacity*(1-maintenance_loss)):
            prod_cost = production_value_unit*(production_day - capacity*(1-maintenance_loss))
            cost += prod_cost
            
        # Additional cost incurred due to performing maintenance too early (we don't want to schedule it more often than needed),
        # when there are still remaining life days
        early_maint_cost = loss_per_life_day*max(life-i[0], 0)
        cost += early_maint_cost
        
        data_cost_maintenance.append((machine, day, cost))
        data_cost_maintenance_detail.append((machine, day, fail_prod_cost, repair_cost, maint_cost, prod_cost, early_maint_cost))
        
        # cost of maintenance/repair for machine "machine" on day "day"
        cost_kpis.append(cost*maintenance[machine, day])
        
# sum over all KPIs and attach the sum to the model object    
cost_kpi = mdl.sum(cost_kpis)
mdl.add_kpi(cost_kpi, "Cost")

df_cost_maintenance = pd.DataFrame(data_cost_maintenance, columns=['machine', 'day', 'cost_maintenance'])
df_cost_maintenance_detail = pd.DataFrame(data_cost_maintenance_detail, columns=['machine', 'day', 'fail_prod_cost', 'repair_cost', 'maint_cost', 'prod_cost', 'early_maint_cost'])

### KPI 2: Planned production

Sum up all the *planned* production numbers provided as input data:

In [43]:
total_planned_production = mdl.sum(df_planned_production.production)
mdl.add_kpi(total_planned_production, "Total Planned Production")


DecisionKPI(name=Total Planned Production,expr=7257)

### KPI 3: Total production

Sum up *actual* production numbers, i.e. the values assigned by the optimization model:

In [44]:
total_production = mdl.sum(df_production.production)
mdl.add_kpi(total_production, "Total Production")

DecisionKPI(name=Total Production,expr=Production_M-01_Day-01+Production_M-01_Day-02+Production_M-01_Da..)

## Objective Function

Here we define our goal, i.e. the metric we are optimizing for. More specifically, we use the "strategy" parameter, specified as part of our model inputs, that defines the optimization metric. Two optimization metrics are provided: 

- If `strategy == 1`, the optimization model will minimize total cost of the solution (**recommended option**)
- Otherwise, the model will try to assign maintenance in such a way that it minimizes the early and late penalties. In other words, we focus on scheduling maintenance on the optimal day for each machine, given when it is likely to fail, while ignoring any potential impact on the cost.

In [45]:
strategy = int(df_parameters[df_parameters.id=='strategy']['value'])

if (strategy == 1):
    mdl.minimize(cost_kpi)
else:
    early = 10
    late = 1000
    maintenance_penalty = []     
    for machine in all_machines:           
        
        last_day = None
        for i, day in np.ndenumerate(all_days):
            last_day = day;
            cumul_failure = int(df_cumul_failure.cumul_failure[machine, day])
            if (cumul_failure > 0):                            
                maintenance_penalty.append(late * maintenance[machine, day] )
            else:
                maintenance_penalty.append(early * i[0] * maintenance[machine, day] )
        
    maintenance_penalty_kpi = mdl.sum(maintenance_penalty)
    mdl.minimize(maintenance_penalty_kpi)


# Solve the model

Now we are ready to solve the model and display the KPIs:

In [46]:
s = mdl.solve(log_output=True)
assert s, "solve failed"
mdl.report()


    

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 2
Found incumbent of value 2556.100000 after 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 100 rows and 150 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.08 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.09 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.09 ticks)
* model PredictiveMaintenance solved with objective = 2556.100
*  KPI: Cost                     = 2556.100
*  KPI: Total Planned Production = 7257.000
*  KPI: Total Production         = 7105.000


# Postprocess the solution and populate output tables

Postprocess solution, generate output tables to be used by the Decision Optimization model `Predictive Maintenance` to display outputs in the dashboard

In [47]:
all_kpis = [(kp.name, kp.compute()) for kp in mdl.iter_kpis()]
df_kpis = pd.DataFrame(all_kpis, columns=['kpi', 'value'])

df_production = df_production.production.apply(lambda v: v.solution_value)
df_maintenance = df_maintenance.maintenance.apply(lambda v: v.solution_value)

df_production = df_production.to_frame()
df_production['machine'] = df_production.index.get_level_values('all_machines') 
df_production['day'] = df_production.index.get_level_values('all_days') 
df_production.columns = ['production', 'machine', 'day'] 

# track production shortage = gap between planned production and actual
prod_shortage = []
for m in all_machines:
   for d in all_days:
      prod_shortage.append((m,d,df_planned_production.production[m,d]-df_production.production[m,d]))

df_production_shortage = pd.DataFrame(prod_shortage, columns=['machine', 'day', 'production_shortage'])
# end track production shortage

df_production = df_production.reset_index(drop=True)

df_maintenance = df_maintenance.to_frame()
df_maintenance['machine'] = df_maintenance.index.get_level_values('all_machines') 
df_maintenance['day'] = df_maintenance.index.get_level_values('all_days') 
df_maintenance.columns = ['maintenance', 'machine', 'day'] 

#track resource load (number of machines maintained each day)
res_load = []
for d in all_days:
    load = 0
    for m in all_machines:
        load += int(df_maintenance.maintenance[m,d])
    res_load.append((d,load))
df_resource_load = pd.DataFrame(res_load, columns=['day', 'load'])
# end track resource load 

df_cumul_failure['machine'] = df_cumul_failure.index.get_level_values('machine')
df_cumul_failure['day'] = df_cumul_failure.index.get_level_values('day')
df_cumul_failure.columns = ['failure', 'machine', 'day']
df_cumul_failure = df_cumul_failure.reset_index(drop=True)

# detailed maintenance cost
df_cost_maintenance_detail['filter'] = df_cost_maintenance_detail.apply(lambda x: df_maintenance.maintenance[x['machine'],x['day']]==1, axis=1)
df_cost_maintenance_detail = df_cost_maintenance_detail[df_cost_maintenance_detail['filter'] == True]
df_cost_maintenance_detail.drop(['filter'], axis=1, inplace=True)

df_maintenance = df_maintenance.reset_index(drop=True)

# maintenance kpi: cost breakdown
df_cost_maint_sum = df_cost_maintenance_detail.drop(['machine', 'day'], axis=1)
cost_maint_vals = df_cost_maint_sum.sum().reset_index().values
df_maint_kpis = pd.DataFrame(cost_maint_vals, columns=['kpi', 'value'])

outputs = {}
    
outputs['cost_maintenance'] = df_cost_maintenance
outputs['cumul_failure'] = df_cumul_failure
outputs["kpis"] = df_kpis
outputs["maint_kpis"] = df_maint_kpis
outputs["maintenance"] = df_maintenance
outputs["production"] = df_production
outputs["production_shortage"] = df_production_shortage
outputs["resource_load"] = df_resource_load
outputs['maintenance_cost_detail'] = df_cost_maintenance_detail

print(df_kpis)

                        kpi   value
0                      Cost  2556.1
1  Total Planned Production  7257.0
2          Total Production  7105.0
